## Assymetric Cryptography

In [1]:
import random
import math
import codecs
import base64

In [2]:
class Utils:
    def __init__(self):
        '''if not used then delete'''
    
    def hex_to_dec(hex):
        deci = int(hex, 16)
        return deci

    def dec_to_hex(dec):
        hexa = hex(dec)
        return hexa

    def hex_to_b64(hex):
        hex = hex[2:] if(len(hex[2:]) % 2 == 0) else '0' + hex[2:]
        b64 = codecs.encode(codecs.decode(hex, 'hex'), 'base64').decode()
        return b64
       
    def b64_to_hex(b64):
        hexa = base64.b64decode(b64).hex()
        return hexa
    
    def dec_to_b64(dec):
        b64 = Utils.hex_to_b64(Utils.dec_to_hex(dec))
        return b64

    def b64_to_dec(b64):
        deci = Utils.hex_to_dec(Utils.b64_to_hex(b64))
        return deci

    def is_square(i):
        return i == math.isqrt(i) ** 2

    def mod_sqrt(n, p):

        points = []
        n = n % p
        for x in range (1, p):
            if ((x ** 2) % p == n) :
                points.append(x)
        return points

        

In [3]:
class ElGamal:
    def __init__(self):
        pass

    def create_key(self, p, g, x):
        y = pow(g, x, p)
        public_key = { 'y': y, 'g': g, 'p': p }
        private_key = { 'x': x, 'p': p }
        return public_key, private_key

    def encrypt(self, message, key, public_key):
        a = pow(public_key['g'], key, public_key['p'])
        b = (pow(public_key['y'], key, public_key['p']) * (message % public_key['p'])) % public_key['p']
        return { 'a': a, 'b': b }

    def decrypt(self, message, private_key):
        return ((message['b'] % private_key['p']) * pow(message['a'], (private_key['p'] - 1 - private_key['x']), private_key['p'])) % private_key['p']

In [4]:
elgamal = ElGamal()

public_key, private_key = elgamal.create_key(2357, 2, 1751)

print("Public Key\t: " + str(public_key))
print("Private Key\t: " + str(private_key))

encrypted = elgamal.encrypt(2035, 1520, public_key)
print("Ciphertext\t: " + str(encrypted))

decrypted = elgamal.decrypt(encrypted, private_key)
print("Plaintext\t: " + str(decrypted))

Public Key	: {'y': 1185, 'g': 2, 'p': 2357}
Private Key	: {'x': 1751, 'p': 2357}
Ciphertext	: {'a': 1430, 'b': 697}
Plaintext	: 2035


In [5]:
class ElGamalMachine:
    def __init__(self):
        self.elgamal = ElGamal()
    
    def is_prime(self, num, test_count=1000):
        if(num == 1):
            return False
        if(test_count >= num):
            test_count = num - 1
        for _ in range(test_count):
            val = random.randint(1, num - 1)
            if(pow(val, num-1, num) != 1):
                return False
        return True
    def create_random_prime(self, bit):
        found = False
        while not found:
            p = random.randint(2**(bit-1)+1, 2**bit-1)
            if self.is_prime(p):
                return p
    
    def create_key(self, bit):
        p = self.create_random_prime(bit)
        g = random.randint(1, p - 1)
        x = random.randint(1, p -2)

        public_key, private_key = self.elgamal.create_key(p, g, x)

        public_key['y'] = Utils.dec_to_b64(public_key['y'])
        public_key['g'] = Utils.dec_to_b64(public_key['g'])
        public_key['p'] = Utils.dec_to_b64(public_key['p'])

        private_key['x'] = Utils.dec_to_b64(private_key['x'])
        private_key['p'] = Utils.dec_to_b64(private_key['p'])

        return public_key, private_key

    def encrypt(self, message, public_key):
        y = Utils.b64_to_dec(public_key['y'])
        g = Utils.b64_to_dec(public_key['g'])
        p = Utils.b64_to_dec(public_key['p'])

        p_key = { 'y': y, 'g': g, 'p': p }

        encrypted = ''
        for m in message:
            k = random.randint(1, p-2)
            res = self.elgamal.encrypt(ord(m), k, p_key)
            a = Utils.dec_to_b64(res['a'])
            b = Utils.dec_to_b64(res['b'])
            encrypted += a + b
        return encrypted

    def decrypt(self, encrypted, private_key):
        x = Utils.b64_to_dec(private_key['x'])
        p = Utils.b64_to_dec(private_key['p'])

        p_key = { 'x': x, 'p': p }
        
        encrypted = encrypted.split('=\n')[:-1]
        encrypted_paired = []
        for i in range(0, len(encrypted), 2):
            a = Utils.b64_to_dec(encrypted[i] + '=\n')
            b = Utils.b64_to_dec(encrypted[i+1] + '=\n')
            encrypted_paired.append({ 'a': a, 'b': b })

        message = ''
        for m in encrypted_paired:
            message += chr(self.elgamal.decrypt(m, p_key))
        return message

In [6]:
machine = ElGamalMachine()

public_key, private_key = machine.create_key(256)

encrypted = machine.encrypt('Aku adalah = x +', public_key)
print(encrypted)
print(machine.decrypt(encrypted, private_key))

JtioAIwCYiqlMeDSQZCjlbR+b1iidMnmCVCuHNm14jc=
xA/36wjQ2MFVzpws1NDEvmyrVZ6AfBGRUhIDBIAhy10=
Kv/2ZuVvNAWiceUDbMEenfhemjYjcGvuHJ+HiJNzZ5w=
j61DJ7WwmpHQw4++ATq9dEAR/nlwNHTEmfuDOEjUmV4=
zuCxVIwMcTWcKvKtooL6F87i5VogRYU+sjN5Yr+1lU8=
sIhKJZV8pnWijZ7pSJhLcluFoM9Q3oYLx9W7agQAX0o=
oOzmvW+gAyUz2XNwr8U5LcIn+0yVoCc2ECWhZYpuoGM=
BMTx81vq4VKhtBu2exeuZjhAgiohqP/S+H9FQaAJQHI=
6aw2M+yjZsK3s6m/RdOQm/A2/kzH+IlzeIjDQx+AADM=
FroUGyov3NGwUHwbnLB5R92RfqYGStGZrejSOJuLqgU=
RSS8IG7Nna/8i8PXwcCnvcCUM/rNN1zfazSUCFCc4BY=
uRU3hw3TST9zqECmNikkbIJ6d/8IebAnahnlIHBsvuU=
vDEFUcrb/Vfa0KVndkOaHGU0c3TWyvNXi/rjyHiQNYw=
pIXuyAhnSpR+EHN7MZl55RXnv6ghSUWp7U6zz2Goc9c=
fI+8PvZ3eStLw1K/t+UdfIagRQxESFWD1LN/yy3uUpE=
SlF6UWfcp9GhdcxK1zStiBRKrXAN/lvKfBkPbCnc+Zg=
6wUVdWWG7sFOGtTu+A9iCT8js3hJTKIkhE30blqzEx0=
EYowdKyX8+9x3Ju7iCAecM2vdGh3l0waGUpF8jmMOIQ=
OCMf+fe+PJcJQetHhdXprdYP5My6kbHnMgBvMbTuBXk=
PH/czbr5bnLSQ8cRr24fVnlcWgp/d7gA25RzhbUyQIo=
eSlTo9Hf19DHrtsUxjoHRhYjevxZzrRF45LmnocWLJw=
1Z7kxicThnPpLnHX7jrTvcJEk3Ri/aHqJc5i627mm2E=
eCeMYOLGOe

In [7]:
class EllipticPoint:
    def __init__(self, x=0, y=0, inf=False):
        self.x = x if not inf else math.inf
        self.y = y if not inf else math.inf
        self.inf = inf

    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def is_infinity(self):
        return self.inf
    
    def mirror(self):
        return EllipticPoint(self.x, -self.y, self.inf)

    def __eq__(self, point):
        if(isinstance(point, EllipticPoint)):
            if(self.inf):
                return self.inf == point.inf
            else:
                return self.x == point.x and self.y == point.y
            
        return False

    def __str__(self):
        return "({0}, {1})".format(self.x, self.y)

In [18]:
class EllipticCurve:
    def __init__(self, a, b, p):
        ''' y^2 = x^3 + ax + b '''
        self.a = a
        self.b = b
        self.p = p
        self.points = []
        if(self.is_singular()):
            print("Error: Curve is Singular!.")
    
    def is_singular(self):
        return((((4 * (self.a ** 3)) + (27 * (self.b ** 2))) % self.p) == 0)

    def get_points(self):
        points = [EllipticPoint(inf=True)]
        
        for x in range(self.p):
            y_square = x ** 3 + self.a * x + self.b

            for point in Utils.mod_sqrt(y_square % self.p, self.p):
                points.append(EllipticPoint(x, point))
        self.points = points 
    
    def get_y_square(self, x):
        y_square = x ** 3 + self.a * x + self.b

        for point in Utils.mod_sqrt(y_square % self.p, self.p):
            return(EllipticPoint(x, point))

    def point_addition(self, P, Q):
        if(P == Q):
            return self.point_multiplication(P)
        else:
            xp = P.get_x(); yp = P.get_y();
            xq = Q.get_x(); yq = Q.get_y();

            if(xp == xq):
                return EllipticPoint(inf=True)
            else:
                m = ((yp-yq) * pow((xp-xq), -1, self.p)) % self.p

                xr = (m ** 2 - xp - xq) % self.p
                yr = (m * (xp - xr) - yp) % self.p
                return EllipticPoint(xr, yr)

    def point_substraction(self, P, Q):
        return self.point_addition(P, Q.mirror())

    def point_multiplication(self, P):
        
        xp = P.get_x(); yp = P.get_y();

        if(yp == 0):
            return EllipticPoint(inf=True)
        else:
            m = ((3 * (xp ** 2) + self.a) * pow((2 * yp), -1, self.p)) % self.p
            xr = (m ** 2 - 2 * xp) % self.p
            yr = (m * (xp - xr) - yp) % self.p
            return EllipticPoint(xr, yr)

    def point_scalar_multiplication(self, P, k):
        if(k == 1):
            return P
        elif(k == 2):
            return self.point_multiplication(P)
        else:
            res = self.point_multiplication(P)
            for i in range(k-2):
                temp = self.point_addition(res, P)
                if(temp.is_infinity()):
                    if(i == k-3):
                        return temp
                else:
                    res = temp
            return res
                

# def recurrent(target, b):
#     if(b == 1):
#         return 3
#     elif(b == 2):
#         return 6
#     else:
#         if(b % 2 == 0):
#             return recurrent(target, b-2) + 6
#         else:
#             return recurrent(target, b-1) + 3
# recurrent(3, 23) 
    

In [19]:
curve = EllipticCurve(-1, 188, 751)
curve.get_points()

# print("N: " + str(len(curve.points)))
# for x in curve.points:
#     print(x)

# print()

# P = EllipticPoint(2, 4)
# Q = EllipticPoint(5, 9)
# print(curve.point_addition(P, Q))
# print(curve.point_multiplication(P))

# print()

for i in range(1, len(curve.points)):
    print("i: " + str(i))
    print(curve.point_scalar_multiplication(P, i))






i: 1
(2, 4)
i: 2
(479, 185)
i: 3
(626, 19)
i: 4
(432, 715)
i: 5
(659, 616)
i: 6
(488, 284)
i: 7
(191, 54)
i: 8
(656, 121)
i: 9
(328, 606)
i: 10
(584, 133)
i: 11
(562, 499)
i: 12
(628, 368)
i: 13
(376, 268)
i: 14
(558, 222)
i: 15
(95, 643)
i: 16
(516, 292)
i: 17
(279, 171)
i: 18
(116, 540)
i: 19
(684, 439)
i: 20
(542, 429)
i: 21
(99, 469)
i: 22
(682, 197)
i: 23
(503, 78)
i: 24
(715, 171)
i: 25
(343, 210)
i: 26
(88, 651)
i: 27
(632, 688)
i: 28
(551, 623)
i: 29
(496, 677)
i: 30
(97, 11)
i: 31
(598, 126)
i: 32
(385, 656)
i: 33
(358, 39)
i: 34
(56, 324)
i: 35
(690, 119)
i: 36
(384, 288)
i: 37
(141, 624)
i: 38
(473, 267)
i: 39
(634, 654)
i: 40
(403, 28)
i: 41
(508, 580)
i: 42
(640, 184)
i: 43
(538, 358)
i: 44
(579, 464)
i: 45
(532, 16)
i: 46
(107, 192)
i: 47
(695, 107)
i: 48
(169, 435)
i: 49
(185, 585)
i: 50
(553, 553)
i: 51
(426, 499)
i: 52
(622, 342)
i: 53
(153, 420)
i: 54
(636, 5)
i: 55
(438, 571)
i: 56
(134, 534)
i: 57
(272, 687)
i: 58
(210, 677)
i: 59
(457, 636)
i: 60
(454, 68)
i: 61
(4

TypeError: pow() 3rd argument not allowed unless all arguments are integers

In [ ]:
M = 11

li = [220, 224, 229]

k = 20

found = False
i = 1
while(not found):
    x = M * k + i
    if x in li:
        found = True
    i = i + 1

print(x)

decoded = math.floor((x-1)/k)
print(decoded)

224
11


In [ ]:
# ALICE
# Kunci Privat = a
# Kunci Publik = PA = a . B

# BOB
# Kunci Privat = b
# Kunci Publik = PB = b . B

# Bilangan Acak k dalam interval [1, p-1]

# Ciphertext(PM) = PC = [(kB), (PM + kPB)]
# b . (kB) 


In [ ]:
curve = EllipticCurve(-1, 16, 29)
curve.get_points()

P = EllipticPoint(5, 7)
Q = EllipticPoint(1, 25)
R = EllipticPoint(6, 20)


print(curve.point_multiplication(P))
print(curve.point_addition(Q, R))


(28, 4)
(23, 26)
